In [ ]:
pip install jupyter_dash

In [ ]:
pip install dash_bootstrap_components

In [2]:
import pandas as pd
import dash
from dash import Dash
from dash import jupyter_dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

In [9]:
df = pd.read_csv(r"C:\Users\joe\Josh-Guo-Data-Science\Foods Project\foods_df.csv")

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.LUMEN])
options_list = list(df.columns[1:])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Ingredients & Cuisines Dashboard", style={"font-size":60, "textAlign":"center"},),),
            ]),
    html.Br(),
    dbc.Row([dbc.Col(html.H1("Cuisine"), width="auto"
                    ),
            dbc.Col(
            dcc.Dropdown(
                id="cuisine_dropdown",
                options=[{'label': option, 'value':option} for option in list(df["cuisine"].unique())],
                searchable=True,
                placeholder="Select A Cuisine",
                        ), align="center"
                    )    
            ]),
    
    dbc.Row([dbc.Col(
             dcc.Graph(id="cuisine_ingredient_usage"),
                    ),
            ]),   
    
    html.Br(),
    # Input for graph with most common cuisines for an ingredient
    dbc.Row([dbc.Col(html.H1("Ingredients"), width="auto"
                    ),
            dbc.Col(
            dcc.Dropdown(
                id="ingredient_dropdown",
                options=[{'label': option, 'value': option} for option in options_list],
                multi=True,
                searchable=True,
                placeholder="Select Any Ingredients",
                        ),
            width=8, align="center"
                    )
            ]),
    # Output for graph with most common cuisines for an ingredient
    dbc.Row([dbc.Col(
            dcc.Graph(id="ingredient_most_used_cuisine"),
                    )
            ])
])
@app.callback(Output(component_id="ingredient_most_used_cuisine", component_property="figure"),
              Input(component_id="ingredient_dropdown", component_property="value"))
def ingredient_top_cuisines(ingredient_list):
    ingredient_list.append("cuisine") # So the cuisine column is inlcluded
    ingredient_df = df[ingredient_list]
    # Using sum to get dishes with all the ingredients. The -1 accounts for "cuisine" being in the list
    ingredient_df = ingredient_df[ingredient_df.iloc[:, 0:-1].sum(axis=1) == len(ingredient_list) - 1]
    ingredient_df = pd.DataFrame(ingredient_df.groupby("cuisine")["cuisine"].count().sort_values(ascending=False))
    # Making the bar chart based on ingredient_df
    fig = px.histogram(ingredient_df, x=ingredient_df.index, y="cuisine", title="Most Common Cuisines for Chosen Ingredients",
                       labels={"index":"Cuisine"}, color_discrete_sequence=["#00d8d8"])
    fig.update_layout(xaxis_title="Cuisines", yaxis_title="Amount of Dishes")
    fig.for_each_trace(lambda t: t.update(hovertemplate=t.hovertemplate.replace("sum of cuisine", "Count")))
    fig.update_traces(marker_line_width=1.2, marker_line_color="black")
    return fig

@app.callback(Output(component_id="cuisine_ingredient_usage", component_property="figure"),
              Input(component_id="cuisine_dropdown", component_property="value"))
def unique_ingredients(cuisine):
    unique_ingredients_list = [] # This list will take in the count of ingredients for each usage amount
    cuisine_dishes = df[df["cuisine"] == cuisine]
    
    for i in range(0, 65, 5):
        ingredients = cuisine_dishes.iloc[:, 1:].sum(axis=0) > i # Returns a series where each ingredient is True/False
        ingredients_count = ingredients.sum() # Summing the Trues gives the amount of ingredients passing the condition
        unique_ingredients_list.append(ingredients_count)
    
    # The dataframe made from this dictionary has the usage amount in one column and the count in the other
    unique_ingredients_dict = {"usage": [str(num) + "+" for num in list(range(0, 65, 5))], "ingredients_count": unique_ingredients_list}
    unique_df = pd.DataFrame(unique_ingredients_dict)
    
    # Making a bar graph from the dataframe
    fig = px.histogram(unique_df, x="usage", y="ingredients_count", title="Unique " + cuisine.title() + " Ingredients By Usage Amount")
    fig.update_layout(xaxis_title="Usage Amount", yaxis_title="Ingredients Count")
    fig.for_each_trace(lambda t: t.update(hovertemplate=t.hovertemplate.replace("sum of ingredients_count", "ingredients")))
    fig.update_traces(marker_line_width=1.2, marker_line_color="black")
    return fig


if __name__ == "__main__":
    app.run_server(jupyter_mode="external")